In [6]:
import cv2
import mediapipe as mp
import time

cap = cv2.VideoCapture(0)


mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

sTime = time.time()

pivot_x = 0
pivot_y = 0

row_list =[]

font = cv2.FONT_HERSHEY_SIMPLEX
org = (185, 50)
color = (0, 0, 255)

n = 5 #5 gestures

num = 1 #iter variable

while (cap.isOpened()):
    ret, img = cap.read()
    img = cv2.resize(img, (960, 540))
    x, y, c = img.shape
    t_elapsed = abs(sTime - time.time())
    if num>n:
        break
    print(t_elapsed,num)
    if t_elapsed==10:
        cv2.putText(img, 'Started', org, font, fontScale=1, thickness=2, color=color)
    if t_elapsed>10:
        cv2.putText(img, 'Recording for gesture {}'.format(num), org, font, fontScale=1, thickness=2, color=color)
    if (t_elapsed-10)//10==num:
        num += 1
        cv2.putText(img, 'Recording for gesture {}'.format(num), org, font, fontScale=1, thickness=2, color=color)
        time.sleep(3)
    cTime = time.time()

    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks!=0:
        if results.multi_hand_landmarks:
            for handlms in results.multi_hand_landmarks:
                for id,lms in enumerate(handlms.landmark):
                    #print(id,lms)
                    h,w,c = img.shape
                    #print(img.shape)
                    cx,cy = int(lms.x*w),int(lms.y*h)
                    #print("id:",id,", x:",cx,", y:",cy)
                    if id==0:
                        pivot_x = int(lms.x * x)
                        pivot_y = int(lms.y * y)
                        mpDraw.draw_landmarks(img, handlms, mpHands.HAND_CONNECTIONS)
                        if t_elapsed>10:
                            row_list.append(str(pivot_x))
                            row_list.append(str(pivot_y))
                    else:
                        lmx = int(lms.x * x)
                        lmy = int(lms.y * y)
                        mpDraw.draw_landmarks(img, handlms, mpHands.HAND_CONNECTIONS)
                        if t_elapsed>10:
                            row_list.append(str(pivot_x-lmx))
                            row_list.append(str(pivot_y-lmy))
                break
        if t_elapsed>10 and results.multi_hand_landmarks!=0:
            with open("landmarks_medium.csv", "a") as f:
                if len(row_list)>0:
                    f.write(",".join(row_list)+","+str(num-1)+"\n")
        row_list=[]

    if cv2.waitKey(1) & 0xFF == ord('q'):
        # if the 'q' is pressed quit.'OxFF' is for 64 bit.[if waitKey==True] is condition
        break
    if ret == True:
        cv2.imshow("result",img)
        pass
cap.release()
cv2.destroyAllWindows()

0.5218017101287842 1
0.6102480888366699 1
0.6681540012359619 1
0.7011153697967529 1
0.7391481399536133 1
0.7735393047332764 1
0.8046681880950928 1
0.8362066745758057 1
0.8695173263549805 1
0.9018642902374268 1
0.9505913257598877 1
1.0066356658935547 1
1.11810302734375 1
1.1655354499816895 1
1.2020092010498047 1
1.2342064380645752 1
1.2737946510314941 1
1.3091115951538086 1
1.3740150928497314 1
1.4327139854431152 1
1.459641456604004 1
1.5217418670654297 1
1.5528604984283447 1
1.5857057571411133 1
1.6232776641845703 1
1.6609230041503906 1
1.6944262981414795 1
1.749549388885498 1
1.7915196418762207 1
1.8339643478393555 1
1.8701808452606201 1
1.8982255458831787 1
1.9247682094573975 1
1.966282606124878 1
2.0218939781188965 1
2.049729824066162 1
2.1121268272399902 1
2.177929639816284 1
2.2449638843536377 1
2.302215576171875 1
2.3519551753997803 1
2.409148931503296 1
2.4496023654937744 1
2.492525339126587 1
2.537065029144287 1
2.5857481956481934 1
2.6358964443206787 1
2.6773061752319336 1
2.7

In [6]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('Five - 5_landmarks.csv')

# Splitting the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop('class_id', axis=1), df['class_id'],
                                                    test_size=0.30, random_state=1)

# Logistic Regression model
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression()
logmodel.fit(x_train, y_train)

# Predicting the test set
prediction = logmodel.predict(x_test)

# For testing accuracy
from sklearn.metrics import confusion_matrix, accuracy_score

# Confusion matrix
accuracy_m = confusion_matrix(y_test, prediction)

# Accuracy score
accuracy_s = accuracy_score(y_test, prediction)
print(f"Confusion Matrix:\n{accuracy_m}")
print(f"Accuracy Score: {accuracy_s}")

# Saving the model using pickle
import pickle

filename = 'gesture_model.sav'
pickle.dump(logmodel, open(filename, 'wb'))  # Save logmodel instead of clf


Confusion Matrix:
[[298   1]
 [  0 298]]
Accuracy Score: 0.998324958123953


c:\Users\Admin\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
import cv2
import mediapipe as mp
import time
import pickle

cap = cv2.VideoCapture(0)


mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

sTime = time.time()

pivot_x = 0
pivot_y = 0

row_list =[]

font = cv2.FONT_HERSHEY_SIMPLEX
org = (185, 50)
color = (0, 0, 255)

model_file_name = 'gesture_model.sav'
model = pickle.load(open(model_file_name, 'rb'))
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils
classID = 0
while (cap.isOpened()):
    ret, img = cap.read()
    img = cv2.resize(img, (960, 540))
    x, y, c = img.shape
    t_elapsed = abs(sTime - time.time())

    if t_elapsed>60:
        break
    cTime = time.time()

    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks!=0:
        if results.multi_hand_landmarks:
            for handlms in results.multi_hand_landmarks:
                for id,lms in enumerate(handlms.landmark):
                    #print(id,lms)
                    h,w,c = img.shape
                    #print(img.shape)
                    cx,cy = int(lms.x*w),int(lms.y*h)
                    #print("id:",id,", x:",cx,", y:",cy)
                    if id == 0:
                        pivot_x = int(lms.x * x)
                        pivot_y = int(lms.y * y)
                        mpDraw.draw_landmarks(img, handlms, mpHands.HAND_CONNECTIONS)
                        row_list.append(pivot_x)
                        row_list.append(pivot_y)
                    else:
                        lmx = int(lms.x * x)
                        lmy = int(lms.y * y)
                        mpDraw.draw_landmarks(img, handlms, mpHands.HAND_CONNECTIONS)
                        row_list.append(pivot_x - lmx)
                        row_list.append(pivot_y - lmy)
                break
            prediction = model.predict([row_list])
            cv2.putText(img, 'Predicted gesture {}'.format(prediction[0]), org, font, fontScale=1, thickness=2, color=color)
        row_list=[]

    if cv2.waitKey(1) & 0xFF == ord('q'):
        # if the 'q' is pressed quit.'OxFF' is for 64 bit.[if waitKey==True] is condition
        break
    if ret == True:
        cv2.imshow("result",img)
        pass
cap.release()
cv2.destroyAllWindows()

c:\Users\Admin\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\Admin\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\Admin\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\Admin\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\Admin\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\Admin\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature nam

### 2 hands

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle

# Load dataset
df = pd.read_csv('main_landmarks.csv')

# Splitting the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(df.drop('class_id', axis=1), df['class_id'],
                                                    test_size=0.30, random_state=1)

# Train the logistic regression model
logmodel = LogisticRegression(max_iter=1000)  # Increase iterations for better convergence
logmodel.fit(x_train, y_train)

# Predicting the test set
prediction = logmodel.predict(x_test)

# Confusion matrix and accuracy score
accuracy_m = confusion_matrix(y_test, prediction)
accuracy_s = accuracy_score(y_test, prediction)
print(f"Confusion Matrix:\n{accuracy_m}")
print(f"Accuracy Score: {accuracy_s}")

# Save the model
filename = 'gesture_model_main.sav'
pickle.dump(logmodel, open(filename, 'wb'))


Confusion Matrix:
[[281   0   0   0   0   0   0   0   0   0   0]
 [  0 330   0   0   0   0   0   0   0   0   0]
 [  0   0 301   1   0   0   0   0   0   0   0]
 [  0   0   0 291   0   0   0   0   0   0   0]
 [  0   0   0   0 291   0   0   0   0   0   0]
 [  0   0   0   0   0 280   1   0   0   0   0]
 [  0   0   0   0   0   0 294   0   0   0   0]
 [  0   0   0   0   0   0   0 282   0   0   0]
 [  0   0   0   0   0   0   0   0 307   0   0]
 [  0   0   0   0   0   0   0   0   1 314   0]
 [  0   0   0   1   0   0   0   0   0   0 267]]
Accuracy Score: 0.9987661937075879


c:\Users\Admin\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [3]:
import cv2
import mediapipe as mp
import time
import pickle

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils
font = cv2.FONT_HERSHEY_SIMPLEX
org = (185, 50)
color = (0, 0, 255)

model = pickle.load(open('gesture_model_main.sav', 'rb'))
row_list = []

while cap.isOpened():
    ret, img = cap.read()
    img = cv2.resize(img, (960, 540))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)
    row_list = []

    if results.multi_hand_landmarks:
        for i, handlms in enumerate(results.multi_hand_landmarks):
            for lm in handlms.landmark:
                h, w, _ = img.shape
                row_list.extend([lm.x * w, lm.y * h])

        if len(results.multi_hand_landmarks) < 2:
            row_list.extend([0] * 42)

        prediction = model.predict([row_list])
        cv2.putText(img, f'Predicted gesture {prediction[0]}', org, font, fontScale=1, color=color, thickness=2)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    if ret:
        cv2.imshow("result", img)

cap.release()
cv2.destroyAllWindows()


c:\Users\Admin\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\Admin\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\Admin\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\Admin\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\Admin\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\Admin\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature nam